In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = (SparkSession.builder.master("local[2]").appName("demo")
         .config(map={
    "spark.driver.memory": "2G",
    "spark.jars.packages": "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1,org.postgresql:postgresql:42.6.0",
    "spark.sql.extensions": "com.datastax.spark.connector.CassandraSparkExtensions",
    "spark.sql.catalog.comwatt": "com.datastax.spark.connector.datasource.CassandraCatalog",
    "spark.sql.catalog.comwatt.spark.cassandra.connection.compression": "LZ4",
    "spark.sql.catalog.comwatt.spark.cassandra.connection.host": "localhost:9042",
}).getOrCreate())

23/12/11 23:29:14 WARN Utils: Your hostname, LTO-CWT060 resolves to a loopback address: 127.0.1.1; using 192.168.0.23 instead (on interface wlp0s20f3)
23/12/11 23:29:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/chen/.ivy2/cache
The jars for the packages stored in: /home/chen/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2325ac30-dee7-44f2-9333-af942ad6ab0d;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/chen/Documents/presentation-meetup-datascience/demo/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in local-m2-cache
	found com.typesafe#config;1.4.1 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.26 in local-m2-cache
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in local-m2-cache
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in local-m2-cache
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.2 in spark-list
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in centr

In [3]:
from demo.adapters.postgres_reader import PostgresClient, PostgresReader

pg_client = PostgresClient(spark, "jdbc:postgresql://localhost:5432/comwatt", "postgres", "demo_password")
PostgresReader(pg_client, "devices")().show(truncate=False)

+----------+------------------------------------+-------+---------+-------------------------+
|device_uid|connected_object_uid                |site_id|type     |label                    |
+----------+------------------------------------+-------+---------+-------------------------+
|0         |05ed9851-5343-47d9-9417-99422184eb29|0      |chauffage|Mon chauffage maison     |
|1         |0fd49130-1973-418f-b101-3682942654e0|1      |chauffage|Le chauffage c'est la vie|
|2         |14766b2f-4d6f-4849-afdf-d74639ac67a4|2      |four     |pour faire du pain       |
+----------+------------------------------------+-------+---------+-------------------------+


In [4]:
from demo.adapters.cassandra_reader import  CassandraClient,CassandraReader

cass_client = CassandraClient(spark, "comwatt.comwatt")
CassandraReader(cass_client, "measures")().show(truncate=False)

+-------+-----------+------------------------------------+-------------------+----------+
|site_id|time_bucket|connected_object_uid                |timestamp          |value     |
+-------+-----------+------------------------------------+-------------------+----------+
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-12 01:00:00|106.82685 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-12 00:00:00|95.88386  |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 23:00:00|123.56678 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 22:00:00|108.77442 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 21:00:00|91.61672  |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 20:00:00|110.88063 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 19:00:00|95.758224 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 18:00:00|95.24462  |
|1      |2